## ISE 535 Final Project

In [361]:
import numpy as np
import pandas as pd
import requests
import plotly.express as px
import plotly.graph_objs as go
import datetime
from datetime import datetime
import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')

from ipywidgets import interact, IntSlider, interactive, widgets, interact_manual, fixed
from pandas import DataFrame
from fbprophet import Prophet

pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',1000)

pd.options.display.float_format = "{:.2f}".format
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
%matplotlib inline

from IPython.display import display, HTML
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Functions


#### 1. List of Countries with Data

In [362]:
def countries():
    url = "https://api.covid19api.com/countries"
    payload={}
    headers = {'X-Access-Token': 'a3c98472-2928-4752-9d71-083ce072213c'}
    response = requests.request("GET", url, headers=headers, data=payload)
    data = response.json()
    df = DataFrame(data).sort_values(by='Country')
    return df

#### 2. Generate All Data of a Country

In [363]:
def countryData(country):
    url1 = "https://api.covid19api.com/dayone/country/"
    country = country
    url = url1+country
    payload={}
    headers = {'X-Access-Token': 'a3c98472-2928-4752-9d71-083ce072213c'}
    response = requests.request("GET", url, headers=headers, data=payload)
    data = response.json()
    df = DataFrame(data)
    return df

#### 3. Generate Latest Data of a Country

In [364]:
def latestCountry(country):
    url1 = "https://api.covid19api.com/dayone/country/"
    country = country
    url = url1+country
    payload={}
    headers = {'X-Access-Token': 'a3c98472-2928-4752-9d71-083ce072213c'}
    response = requests.request("GET", url, headers=headers, data=payload)
    data = response.json()
    df = DataFrame(data)
    return df.tail(1)

## Data Generation and  Wrangling 

#### 1. List of Countries that have Covid-19 Data from the API

In [365]:
dfCountries = countries()
dfCountries

,Country,Slug,ISO2
107,ALA Aland Islands,ala-aland-islands,AX
41,Afghanistan,afghanistan,AF
83,Albania,albania,AL
64,Algeria,algeria,DZ
214,American Samoa,american-samoa,AS
6,Andorra,andorra,AD
2,Angola,angola,AO
238,Anguilla,anguilla,AI
99,Antarctica,antarctica,AQ
74,Antigua and Barbuda,antigua-and-barbuda,AG


#### 2. Latest Cases for South East Asian Countries

In [ ]:
seCountries = ["brunei","myanmar","cambodia","timor-leste","indonesia","malaysia", 
               "philippines","singapore","thailand","vietnam"]
dataSE = []

for country in seCountries:
    df = latestCountry(country)
    columns = list(df)
    values = df.iloc[0]
    zipped = zip(columns, values)
    a_dictionary = dict(zipped)
    dataSE.append(a_dictionary)

dataSE = DataFrame(dataSE).sort_values('Country')
dataSE['Lat'] = pd.to_numeric(dataSE['Lat'])
dataSE['Lon'] = pd.to_numeric(dataSE['Lon'])
dataSE['Date'] = pd.to_datetime(dataSE.Date, format='%Y-%m-%d %H:%M:%S')
dataSE['Date'] = dataSE['Date'].dt.strftime('%Y-%m-%d')

dataSE.head()

#### 3. All Data for All South East Asian Countries

In [ ]:
bigData = []
for country in seCountries:
    df = countryData(country)
    bigData.append(df)
bigData = pd.concat(bigData)

#cleaning and filtering for widgets use
bigData = bigData[['Country', 'CountryCode','Confirmed','Deaths','Recovered','Active','Date']]
bigData['Date'] = pd.to_datetime(bigData.Date, format='%Y-%m-%d %H:%M:%S')
bigData['Date'] = bigData['Date'].dt.strftime('%Y-%m-%d')
bigData['Date']= pd.to_datetime(bigData["Date"])
bigData = bigData.reset_index(drop=True)

## Visualizations with Widgets

#### 1. Widgets

In [ ]:
layout = widgets.Layout(width='300px', height='30px')
countryFC = widgets.Text(
    value='Philippines',
    placeholder='Philippines',
    description='Type-in Country:',
    layout=layout,
    disabled=False,
    style= {'description_width': 'initial'}
)

varFC = widgets.RadioButtons(
    options=['Confirmed', 'Deaths', 'DailyCases', 'DailyDeaths'],
    value='DailyCases',
    description='Click Variable:',
    disabled=False,
    style= {'description_width': 'initial'}
)

daysFC = widgets.Dropdown(
    options=['7', '14', '30', '60'],
    value='7',
    description='Days to Predict:',
    disabled=False,
    style= {'description_width': 'initial'}
)

variableSel = widgets.RadioButtons(
    options=['Confirmed', 'Deaths', 'Active'],
    value='Confirmed',
    description='Variable:',
    disabled=False
)

countrySel = widgets.Dropdown(
    options=['Brunei Darussalam', 'Cambodia', 'Indonesia','Malaysia','Myanmar',
            'Philippines', 'Singapore', 'Thailand', 'Timor-Leste','Viet Nam'],
    value='Brunei Darussalam',
    description='Country:',
    disabled=False,
)

variableSel = widgets.Dropdown(
    options=['Confirmed', 'Deaths', 'Recovered', 'Active'],
    value='Confirmed',
    description='Variable:',
    disabled=False
)

#Dates entered are exclusive
import datetime
start = datetime.datetime(2020, 1, 1)
end = datetime.datetime(2021, 12, 30)
startDateSel = widgets.DatePicker(description='Start Date:', disabled=False, value=start)
endDateSel = widgets.DatePicker(description='End Date:', disabled=False, value=end)

#### 2. Table of Latest Cases

In [ ]:
#how to select columns and highlight dataframe using gradient background
dataFilter = dataSE[['Country','Confirmed','Deaths', 'Active']]
dataFilter = dataFilter.sort_values('Confirmed', ascending=False).reset_index(drop=True)
dataFilter.style.background_gradient(cmap='Reds')

#### 3. Five Number Summaries of Cases

#### 4. Scatter Plot of Latest Cases per Country

In [ ]:
def scatterLatest(variable):
    fig = px.scatter(dataSE, x='Country', y=variable, size=variable,
                color='Country', hover_name='Country', size_max=50,
                title=f"Latest Covid-19 {variable} Cases as of {dataSE['Date'][0]}")
    return fig.show()

interact(scatterLatest,variable=variableSel)

#### 5. Density Map of Latest Cases per County

In [ ]:
def denseMap(variable): 
    fig = px.density_mapbox(dataSE, lat = 'Lat', lon = 'Lon', hover_name = 'Country',
                       hover_data = [variable], color_continuous_scale='RdBu',
                            radius=50, zoom=3, height=600, z=variable)
    fig.update_layout(title = f"Latest Covid-19 {variable} Cases as of {dataSE['Date'][0]}",
                 mapbox_style = 'open-street-map', mapbox_center_lon = 114)
    return fig.show()

interact(denseMap,variable=variableSel)

#### 6. Plot of Timeseries Cases per Country

In [ ]:
def filter(country, variable, startDate, endDate):
    filter1 = bigData[bigData['Country'] == countrySel.value]
    filter2 = filter1[['Country', 'CountryCode', variableSel.value, 'Date']]
    filter3 = filter2[(filter2['Date']>pd.to_datetime(startDateSel.value)) & 
                   (filter2['Date']<pd.to_datetime(endDateSel.value))]
    fig = px.line(filter3, x='Date', y=[variable],
                 title=f"Covid-19 {variable} Cases of {country}")
    return display(filter3.head()), fig.show()

widget = interact(filter, country=countrySel, variable=variableSel, startDate=startDateSel, endDate=endDateSel)

### Forecasting

#### Step 1: Run the code then type-in the country to be forecasted.

In [ ]:
countryFC

#### Step 2: Run the code then select the variable to be forecasted.

In [ ]:
varFC

#### Step 3: Run the code then choose number of days to predict.

In [ ]:
daysFC

#### Step 4: Run code to generate data from Covid API.

In [ ]:
data = countryData(countryFC.value)
data = data.iloc[:-1 , :]

#Create daily cases
data['DailyCases'] = data['Confirmed'].diff()
data = data.dropna()
data['DailyCases'] = data['DailyCases'].astype(int)

#Create daily deaths
data['DailyDeaths'] = data['Deaths'].diff()
data = data.dropna()
data['DailyDeaths'] = data['DailyDeaths'].astype(int)

#show tail of data
data.tail()

#### Step 5: Run code to calculate Quantitative Summaries of the Variable

In [ ]:
table = data[varFC.value].describe().to_frame()
display(HTML(table.to_html()))

#### Step 6: Run code to plot data timeseries.

In [ ]:
fig = px.line(data, x=data['Date'], y=data[varFC.value],
             title=f"Covid-19 {varFC.value} Cases of {countryFC.value}")
fig.show()

#### Step 6. Run code to forecast cases for the next few days using machine learning.

In [ ]:
#prepare data
df = data[[varFC.value, 'Date']]
df['Date'] = pd.to_datetime(df.Date, format='%Y-%m-%d %H:%M:%S')
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
df['Date']= pd.to_datetime(df["Date"])
df.columns = ['y', 'ds']

#fit model using Prophet
m = Prophet(interval_width=0.95, yearly_seasonality=True)
model = m.fit(df)

#generate predictions using model fit
future = m.make_future_dataframe(periods=int(daysFC.value), freq='D')
forecast = m.predict(future)

#print prediction of the next few days
forecast[['ds','yhat','yhat_lower','yhat_upper']].tail(int(daysFC.value))

#### Step 7: Run code to plot actual and predicted values with corresponding error bands.

In [ ]:
fig = go.Figure([
    go.Scatter(
        name='Forecast',
        x=forecast['ds'],
        y=forecast['yhat'],
        mode='lines',
        line=dict(color='rgb(31, 119, 180)'),
    ),
    go.Scatter(
        name='Upper Bound',
        x=forecast['ds'],
        y=forecast['yhat_upper'],
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=1),
        showlegend=False
    ),
    go.Scatter(
        name='Lower Bound',
        x=forecast['ds'],
        y=forecast['yhat_lower'],
        marker=dict(color="#444"),
        line=dict(width=1),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        showlegend=False
    ),
    go.Scatter(
        x=df['ds'], 
        y = df['y'], 
        mode='markers', 
        name='Actual'
    )
])
fig.update_layout(
    yaxis_title=f'{varFC.value} Cases',
    title=f'Actual and Forecasted {varFC.value} Cases for {countryFC.value}',
    hovermode="x"
)
fig.show()

In [ ]:
#There are built-in plots in the prophet package that can be used to understand trends.
plot2 = m.plot_components(forecast)

#### Step 7: Run code to test performance of model via cross validation.

In [ ]:
#from prophet.diagnostics import cross_validation
#df_cv = cross_validation(m, initial='570 days', period='7 days', horizon = '30 days')

In [ ]:
#from prophet.diagnostics import performance_metrics
#df_p = performance_metrics(df_cv)
#df_p.head()